# Different Models for Different Users
## Based on User Profile Length

In [ ]:
def init_kaggle(run_on_kaggle, secret_name):
    if run_on_kaggle:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        personal_token = user_secrets.get_secret(secret_name)
        
        import subprocess
        subprocess.run(["git", "clone", "https://" + personal_token + "@github.com/alecontuIT/rec_sys_challenge2022.git"])
        
        import os
        os.chdir("./rec_sys_challenge2022")
        
        subprocess.run(["pip", "install", "-r", "requirements.txt"])
        return True
    
    else:
        return False

In [ ]:
kaggle = init_kaggle(False, "recsys_git_token")

In [ ]:
import utils
from recmodels import *
from Recommenders.UserProfileRecommender.UserProfileRecommender import UserProfileRec
from Evaluation.Evaluator import EvaluatorHoldout

In [ ]:
recommender_class = UserProfileRec
dataset_version = "interactions-all-ones"

In [ ]:
num_groups = 20
cf_rec_classes = [P3AlphaRec, UserKNNCFRec, ItemKNNCFRec, RP3BetaRec, TopPopRec, EASE_R_Rec]#, IALSRec, SLIM_BPRRec]
cb_rec_classes = []#ItemKNNCBFRec]

In [ ]:
URM_all, URM_train, URM_validation, ICM = utils.get_data_global_sample(dataset_version=dataset_version, 
                                                                       train_percentage=0.7,
                                                                       setSeed=True)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [ ]:
recommender = UserProfileRec(URM_all, ICM, cf_rec_classes, cb_rec_classes, dataset_version)
recommender.fit(num_groups)

In [ ]:
recommender.save_model(recommender, 
                       utils.get_hyperparams_search_output_folder(recommender_class, 
                                                                  dataset_version=dataset_version))

In [ ]:
prediction = utils.submission(recommender, dataset_version)